# Week 3 Graded Assignment

## Start of Part 1 of the assignment

In case lxml does not exist, we need to install it and then restart the kernel

In [68]:
!pip install lxml

In [69]:
import lxml
import pandas as pd

PostalCodeTable = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
CanadaPostalCode = PostalCodeTable[0]
CanadaPostalCode

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West
285,M8Z,Etobicoke,South of Bloor


Let us start cleaning the data 

1. Remove all rows with Borough = 'Not assigned' 

In [70]:
CanadaPostalCode.drop(CanadaPostalCode[CanadaPostalCode.Borough == 'Not assigned'].index, inplace=True)
CanadaPostalCode

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
...,...,...,...
281,M8Z,Etobicoke,Kingsway Park South West
282,M8Z,Etobicoke,Mimico NW
283,M8Z,Etobicoke,The Queensway West
284,M8Z,Etobicoke,Royal York South West


2. Set the 'Not assigned' neighborhoods, the same name as its relative Borough

In [71]:
# Let us check if this condition exists
EmptyNeighborhood = CanadaPostalCode.loc[CanadaPostalCode.Neighborhood == 'Not assigned']
EmptyNeighborhood

,Postcode,Borough,Neighborhood
9,M9A,Queen's Park,Not assigned


In [72]:
# One row exists so we need to fix it by assigning the neighborhood as the borough name
CanadaPostalCode.loc[(CanadaPostalCode.Neighborhood == 'Not assigned'),'Neighborhood'] = CanadaPostalCode.Borough

In [73]:
# Let us check if it still exists
EmptyNeighborhood = CanadaPostalCode.loc[CanadaPostalCode.Neighborhood == 'Not assigned']
EmptyNeighborhood

,Postcode,Borough,Neighborhood


An empty result which means the neighborhood of Postcode M9A has been edited

3. Combine rows with the same postcode

In [74]:
CombinedPostalCode = CanadaPostalCode.groupby(['Postcode','Borough'])['Neighborhood'].apply(','.join).reset_index()
CleanCanadaPostalCode = pd.DataFrame(CombinedPostalCode)
CleanCanadaPostalCode.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [75]:
print('The final list has {} rows'.format(CleanCanadaPostalCode.shape[0]))

The final list has 103 rows


## Start of Part 2 of the assignment

To avoid any discrepancies or wrong results for the coordinates due to the issues with the geocoder package, we will depend on the csv file provided
for this assignment

In [76]:
Coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
Coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now we have the cleanCanadaPostalCode dataframe and the Coordinates dataframe. We will merge them together to insure that each Postcode has its correct 
Latitude and Longitude. The comparison will be done on Postcode from the cleanCanadaPostalCode equals the value of Postal Code from the coordinates dataframe.
Once the merge is done, we will have an extra column of Postal Code which is the same as Postcode, so we remove this duplicate column

In [77]:
CanadaNeighborhoods = pd.merge(CleanCanadaPostalCode, Coordinates, left_on='Postcode', right_on='Postal Code')
CanadaNeighborhoods.drop(['Postal Code'], axis=1, inplace=True)
CanadaNeighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437
